### 🔍 Search and list all Notion databases your integration can access — shows title, ID, and URL for each

In [ ]:
from notion_client import Client

notion = Client(auth="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")  # paste your key here

# Search all databases your integration can see
results = notion.search(filter={"property": "object", "value": "database"})

print("🔍 Accessible Notion Databases:")
for result in results['results']:
    print(f"- Title: {result['title'][0]['plain_text'] if result['title'] else '[Untitled]'}")
    print(f"  ID: {result['id']}")
    print(f"  URL: {result['url']}")
    print("-----")

### 🧾 Inspect property keys and types for the first page in a Notion database

In [2]:
from notion_client import Client

notion = Client(auth="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")

database_id = "1ad44833-084d-801b-9107-e32cfb219f33"
response = notion.databases.query(database_id=database_id)

first_page = response["results"][0]
props = first_page["properties"]

print("🔍 Property Keys & Types:")
for key, val in props.items():
    print(f"- {key} ({val['type']})")


🔍 Property Keys & Types:
- Is To Do? (formula)
- Tags (multi_select)
- Feature request response (rich_text)
- Group_ID (rich_text)
- Reporter (created_by)
- Area (relation)
- Status (status)
- Last Update (last_edited_time)
- Go Home (button)
- Ranking (rich_text)
- Cost (rich_text)
- Assignee (people)
- Created (created_time)
- Release (relation)
-   (unique_id)
- Is In Progress? (formula)
- Priority (select)
- Feature request description (rich_text)
- Estimate (rich_text)
- Separator (rich_text)
- Is Complete? (formula)
- Severity (select)
- Attachments (files)
- Environment (relation)
- Is Testing Complete? (formula)
- Expected Changes (rich_text)
- Is Failed? (formula)
- General Notes (rich_text)
- Title (title)


### 📤 Export all pages from a Notion database to CSV with selected fields and property handlers

In [ ]:
from notion_client import Client
import csv

notion = Client(auth="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")  # Replace with your API key

database_id = "1ad44833-084d-801b-9107-e32cfb219f33"
response = notion.databases.query(database_id=database_id)

def get_rich_text(prop):
    return " ".join([t["plain_text"] for t in prop]) if prop else ""

def get_people(prop):
    return ", ".join([p["name"] for p in prop]) if prop else ""

def get_select(prop):
    return prop["name"] if prop else ""

def get_multi_select(prop):
    return ", ".join([tag["name"] for tag in prop]) if prop else ""

def get_relation_ids(prop):
    return ", ".join([rel["id"] for rel in prop]) if prop else ""

def get_files(prop):
    return ", ".join([f["file"]["url"] if "file" in f else f["external"]["url"] for f in prop]) if prop else ""

# CSV output
with open("feature_requests_full_export.csv", mode="w", newline="", encoding="utf-8") as csv_file:
    writer = csv.writer(csv_file)

    # Header
    writer.writerow([
        "Title", "Description", "Expected Changes", "General Notes", "Feature Request Response",
        "Assignee", "Reporter", "Status", "Priority", "Severity", "Tags",
        "Environment", "Area", "Release",
        "Is To Do?", "Is In Progress?", "Is Complete?", "Is Failed?", "Is Testing Complete?",
        "Created", "Last Update", "Attachments"
    ])

    for page in response["results"]:
        props = page["properties"]

        writer.writerow([
            get_rich_text(props["Title"]["title"]),
            get_rich_text(props["Feature request description"]["rich_text"]),
            get_rich_text(props["Expected Changes"]["rich_text"]),
            get_rich_text(props["General Notes"]["rich_text"]),
            get_rich_text(props["Feature request response"]["rich_text"]),
            get_people(props["Assignee"]["people"]),
            props["Reporter"]["created_by"]["name"] if props["Reporter"]["created_by"] else "",
            get_select(props["Status"]["status"]),
            get_select(props["Priority"]["select"]),
            get_select(props["Severity"]["select"]),
            get_multi_select(props["Tags"]["multi_select"]),
            get_relation_ids(props["Environment"]["relation"]),
            get_relation_ids(props["Area"]["relation"]),
            get_relation_ids(props["Release"]["relation"]),
            props["Is To Do?"]["formula"]["boolean"],
            props["Is In Progress?"]["formula"]["boolean"],
            props["Is Complete?"]["formula"]["boolean"],
            props["Is Failed?"]["formula"]["boolean"],
            props["Is Testing Complete?"]["formula"]["boolean"],
            props["Created"]["created_time"],
            props["Last Update"]["last_edited_time"],
            get_files(props["Attachments"]["files"]),
        ])

print("✅ Export complete: feature_requests_full_export.csv")


### 🔁 Sync Notion DB from CSV with backup, updates, and new page creation (rate-limited)

In [ ]:
from notion_client import Client
import pandas as pd
import time

# 🔐 Configure
NOTION_TOKEN = "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"
DATABASE_ID = "1ad44833-084d-801b-9107-e32cfb219f33"

notion = Client(auth=NOTION_TOKEN)

# 🔄 Load CSV
csv_data = pd.read_csv("feature_requests_full_export2.csv").fillna("")

# 🔄 Fetch current Notion DB contents (for backup + lookup)
notion_pages = []
start_cursor = None

while True:
    query = notion.databases.query(database_id=DATABASE_ID, start_cursor=start_cursor)
    notion_pages.extend(query["results"])
    if not query.get("has_more"):
        break
    start_cursor = query["next_cursor"]

# 🧠 Build Notion Title Lookup
notion_lookup = {
    page["properties"]["Title"]["title"][0]["plain_text"]: page
    for page in notion_pages
    if page["properties"]["Title"]["title"]
}

# 🧷 Backup current Notion data
def safe_rich_text(props, field):
    try:
        return " ".join([r["plain_text"] for r in props.get(field, {}).get("rich_text", [])])
    except Exception:
        return ""

def safe_select(props, field):
    select_value = props.get(field, {}).get("select")
    return select_value.get("name") if select_value else ""

def safe_status(props, field):
    status_value = props.get(field, {}).get("status")
    return status_value.get("name") if status_value else ""

backup = []
for title, page in notion_lookup.items():
    props = page.get("properties", {})
    backup.append({
        "Title": title,
        "Status": safe_status(props, "Status"),
        "Description": safe_rich_text(props, "Feature request description"),
        "Expected Changes": safe_rich_text(props, "Expected Changes"),
        "General Notes": safe_rich_text(props, "General Notes"),
        "Feature Request Response": safe_rich_text(props, "Feature request response"),
    })

pd.DataFrame(backup).to_csv("notion_feature_requests_backup.csv", index=False)

# 🔁 Start Sync
def text_block(text):
    return [{"type": "text", "text": {"content": text}}] if text else []

created, updated, skipped = 0, 0, 0

for _, row in csv_data.iterrows():
    title = row["Title"]
    existing = notion_lookup.get(title)

    properties = {
        "Title": {"title": text_block(row["Title"])}
    }

    # Only add fields that exist in CSV
    if "Description" in row and pd.notna(row["Description"]):
        properties["Feature request description"] = {"rich_text": text_block(row["Description"])}

    if "Expected Changes" in row and pd.notna(row["Expected Changes"]):
        properties["Expected Changes"] = {"rich_text": text_block(row["Expected Changes"])}

    if "Feature Request Response" in row and pd.notna(row["Feature Request Response"]):
        properties["Feature request response"] = {"rich_text": text_block(row["Feature Request Response"])}

    if "General Notes" in row and pd.notna(row["General Notes"]):
        properties["General Notes"] = {"rich_text": text_block(row["General Notes"])}

    if "Status" in row and pd.notna(row["Status"]):
        properties["Status"] = {"status": {"name": row["Status"]}}

    if "Priority" in row and pd.notna(row["Priority"]):
        properties["Priority"] = {"select": {"name": row["Priority"]}}

    if "Severity" in row and pd.notna(row["Severity"]):
        properties["Severity"] = {"select": {"name": row["Severity"]}}

    if "Tags" in row and pd.notna(row["Tags"]):
        tag_list = [tag.strip() for tag in row["Tags"].split(",") if tag.strip()]
        properties["Tags"] = {"multi_select": [{"name": tag} for tag in tag_list]}

    # 🧹 Remove any None properties
    properties = {k: v for k, v in properties.items() if v is not None}

    # 🚀 Sync with Notion
    print(f"🔍 Processing row: {title}")
    print(f"   → Exists in Notion? {'Yes' if existing else 'No'}")
    try:

        if existing:
            page_id = existing["id"]
            notion.pages.update(page_id=page_id, properties=properties)
            print(f"🔄 Updated: {title}")
            updated += 1
        else:
            notion.pages.create(parent={"database_id": DATABASE_ID}, properties=properties)
            print(f"➕ Created: {title}")
            created += 1
    except Exception as e:
        print(f"❌ Failed to sync '{title}': {e}")
        skipped += 1

    time.sleep(0.4)  # ⏱ Respect Notion API rate limit

print(f"\n✅ Sync Summary: {created} created, {updated} updated, {skipped} skipped.")


### 🔃 Bidirectional Notion ↔ CSV sync with interactive UI and backup support


In [ ]:
from notion_client import Client
import pandas as pd
import time
import sys
import os

# 🔐 Configure
NOTION_TOKEN = "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"
DATABASE_ID = "1ad44833-084d-801b-9107-e32cfb219f33"
CSV_FILE = "Canfor_Integration.csv"
BACKUP_FILE = "notion_feature_requests_backup.csv"

notion = Client(auth=NOTION_TOKEN)

def text_block(text):
    return [{"type": "text", "text": {"content": text}}] if text else []

def safe_rich_text(props, field):
    try:
        return " ".join([r["plain_text"] for r in props.get(field, {}).get("rich_text", [])])
    except Exception:
        return ""

def safe_select(props, field):
    select_value = props.get(field, {}).get("select")
    return select_value.get("name") if select_value else ""

def safe_status(props, field):
    status_value = props.get(field, {}).get("status")
    return status_value.get("name") if status_value else ""

def fetch_notion_pages():
    pages = []
    start_cursor = None
    while True:
        query = notion.databases.query(database_id=DATABASE_ID, start_cursor=start_cursor)
        pages.extend(query["results"])
        if not query.get("has_more"):
            break
        start_cursor = query["next_cursor"]
    return pages

def notion_to_csv():
    print("📥 Exporting Notion → CSV...")
    pages = fetch_notion_pages()
    notion_lookup = {
        page["properties"]["Title"]["title"][0]["plain_text"]: page
        for page in pages if page["properties"]["Title"]["title"]
    }

    backup = []
    for title, page in notion_lookup.items():
        props = page.get("properties", {})
        backup.append({
            "Title": title,
            "Status": safe_status(props, "Status"),
            #"Priority": safe_select(props, "Priority"),
            #"Severity": safe_select(props, "Severity"),
            "Description": safe_rich_text(props, "Feature request description"),
            "Expected Changes": safe_rich_text(props, "Expected Changes"),
            "Feature Request Response": safe_rich_text(props, "Feature request response"),
            "General Notes": safe_rich_text(props, "General Notes"),
            "Estimate": safe_rich_text(props, "Estimate"),
            "Cost": safe_rich_text(props, "Cost"),
            "Tags": ", ".join(t.get("name") for t in props.get("Tags", {}).get("multi_select", []))
        })

    pd.DataFrame(backup).to_csv(BACKUP_FILE, index=False)
    print(f"✅ Notion data exported to `{BACKUP_FILE}`")


def csv_to_notion():
    print("📤 Importing CSV → Notion...")
    print(f"📂 Checking file: {CSV_FILE}")
    print(f"🧪 File exists? {os.path.isfile(CSV_FILE)}")
    print(f"📍 Working directory: {os.getcwd()}")

    if not os.path.isfile(CSV_FILE):
        print(f"❌ File `{CSV_FILE}` does not exist on disk.")
        return

    # Try reading CSV with UTF-8, fallback to ISO-8859-1
    try:
        print(f"📄 Loading CSV from `{CSV_FILE}`...")
        try:
            csv_data = pd.read_csv(CSV_FILE, encoding="utf-8").fillna("")
            print(f"✅ Loaded {len(csv_data)} rows using UTF-8.")
        except UnicodeDecodeError as e:
            print(f"⚠️ UTF-8 decode failed: {e}")
            csv_data = pd.read_csv(CSV_FILE, encoding="ISO-8859-1").fillna("")
            print(f"✅ Loaded {len(csv_data)} rows using ISO-8859-1.")
    except Exception as e:
        print(f"❌ Failed to load CSV: {e}")
        return

    # Fetch existing Notion pages
    pages = fetch_notion_pages()
    notion_lookup = {
        page["properties"]["Title"]["title"][0]["plain_text"]: page
        for page in pages if page["properties"]["Title"]["title"]
    }

    created, updated, skipped = 0, 0, 0

    # Iterate rows in CSV
    for _, row in csv_data.iterrows():
        title = row["Title"]
        existing = notion_lookup.get(title)

        print(f"🔍 Processing row: {title}")
        properties = {
            "Title": {"title": text_block(title)}
        }

        # Rich text fields
        for field in [
            "Description", "Expected Changes", "Feature Request Response", "General Notes",
            "Estimate", "Cost"
        ]:
            notion_field = {
                "Description": "Feature request description",
                "Feature Request Response": "Feature request response"
            }.get(field, field)

            if field in row and pd.notna(row[field]):
                properties[notion_field] = {"rich_text": text_block(row[field])}

        # Status field
        if "Status" in row and pd.notna(row["Status"]):
            properties["Status"] = {"status": {"name": row["Status"]}}

        # Priority field
        if "Priority" in row and pd.notna(row["Priority"]):
            properties["Priority"] = {"select": {"name": row["Priority"]}}

        # Severity field
        if "Severity" in row and pd.notna(row["Severity"]):
            properties["Severity"] = {"select": {"name": row["Severity"]}}

        # Tags field
        if "Tags" in row and pd.notna(row["Tags"]):
            tags = [t.strip() for t in row["Tags"].split(",") if t.strip()]
            properties["Tags"] = {"multi_select": [{"name": tag} for tag in tags]}

        try:
            if existing:
                notion.pages.update(page_id=existing["id"], properties=properties)
                print(f"🔄 Updated: {title}")
                updated += 1
            else:
                notion.pages.create(parent={"database_id": DATABASE_ID}, properties=properties)
                print(f"➕ Created: {title}")
                created += 1
        except Exception as e:
            print(f"❌ Failed to sync '{title}': {e}")
            skipped += 1

        time.sleep(0.4)

    print(f"\n✅ CSV Sync Done: {created} created, {updated} updated, {skipped} skipped.")

# 🧠 Main Entry
def run_sync(choice):
    if choice == "1":
        notion_to_csv()
    elif choice == "2":
        csv_to_notion()
    elif choice == "3":
        notion_to_csv()
        csv_to_notion()
    else:
        print("❌ Invalid option.")

def is_running_in_notebook():
    try:
        from IPython import get_ipython
        return get_ipython() is not None
    except ImportError:
        return False

def launch_interface():
    if is_running_in_notebook():
        from ipywidgets import Dropdown, Button, VBox, Output, HBox
        from IPython.display import display

        options = {
            "Export Notion → CSV": "1",
            "Import CSV → Notion": "2",
            "Full Sync (both directions)": "3"
        }

        dropdown = Dropdown(
            options=options,
            description="Sync Mode:",
            style={'description_width': 'initial'},
            layout={'width': '300px'}
        )

        button = Button(
            description="Run Sync",
            button_style='success',
            tooltip="Click to begin sync",
            layout={'width': '150px'}
        )

        out = Output()

        def on_click(b):
            with out:
                out.clear_output()
                choice = dropdown.value
                print(f"🔃 Running '{dropdown.label}'...\n")
                run_sync(choice)

        button.on_click(on_click)

        display(VBox([HBox([dropdown, button]), out]))

    else:
        print("🔁 Notion Sync Utility")
        print("1 - Export Notion → CSV")
        print("2 - Import CSV → Notion")
        print("3 - Full Sync (both directions)")
        choice = input("Select an option (1, 2, or 3): ").strip()
        run_sync(choice)


if __name__ == "__main__" or is_running_in_notebook():
    launch_interface()

### 🔃 Bidirectional Notion ↔ CSV sync with interactive UI and backup support

> **Note:**
> • Select between export, import, or full sync using a dropdown UI (in notebooks) or CLI prompt.
> • Automatically backs up existing Notion data to CSV before changes.
> • Handles text, status, select, multi-select, and tag fields.
> • Gracefully falls back from UTF-8 to ISO-8859-1 if CSV encoding fails.
> • Adds a 0.4s delay per request to respect Notion API rate limits.